In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import t

# Настройка визуализации
sns.set(style="whitegrid")


In [ ]:
df = pd.read_csv('/Users/liker/Downloads/students_simple.csv')
df.head()


In [ ]:
columns_to_plot = ['height', 'age']
df_selected = df[columns_to_plot]
df_selected.head()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i, col in enumerate(columns_to_plot):
    sns.histplot(df_selected[col], bins=20, kde=True, ax=axes[i])
    axes[i].set_title(f'Гистограмма столбца {col}')

plt.tight_layout()
plt.show()

In [ ]:
# Построение scatter plot между двумя столбцами
plt.figure(figsize=(7,5))
sns.scatterplot(data=df_selected, x=columns_to_plot[0], y=columns_to_plot[1])
plt.title(f'Scatter plot: {columns_to_plot[0]} vs {columns_to_plot[1]}')
plt.xlabel(columns_to_plot[0])
plt.ylabel(columns_to_plot[1])
plt.show()


In [ ]:
x = df_selected['height']
y = df_selected['age']
x_mean = x.mean()
y_mean = y.mean()

# Вычисление знаков отклонений
sign_x = np.sign(x - x_mean)
sign_y = np.sign(y - y_mean)

# Совпадения и несовпадения знаков
same_sign = (sign_x == sign_y).sum()
diff_sign = (sign_x != sign_y).sum()

# Корреляция Фехнера
n = len(df_selected)
fechner_corr = (same_sign - diff_sign) / n

print(f"Корреляция Фехнера между height и age: {fechner_corr:.4f}")


In [ ]:
import numpy as np
from scipy.stats import pearsonr, norm

# Выбираем два столбца
x = df_selected[columns_to_plot[0]]
y = df_selected[columns_to_plot[1]]

# Корреляция Пирсона + p-value
r, p_value = pearsonr(x, y)
print(f"Корреляция Пирсона между {columns_to_plot[0]} и {columns_to_plot[1]}: {r:.3f}, p-value: {p_value:.3f}")

# Средние значения
x_mean = x.mean()
y_mean = y.mean()

# Отклонения от среднего
x_dev = x - x_mean
y_dev = y - y_mean

# Числитель: сумма произведений отклонений
numerator = np.sum(x_dev * y_dev)

# Знаменатель: корень из произведения квадратов отклонений
denominator = np.sqrt(np.sum(x_dev**2) * np.sum(y_dev**2))

# Коэффициент корреляции Пирсона
pearson_corr = numerator / denominator

# Статистика t
t_stat = pearson_corr * np.sqrt(n - 2) / np.sqrt(1 - pearson_corr**2)

# p-value (двусторонний тест)
p_value = 2 * t.sf(np.abs(t_stat), df=n - 2)

print(f"Корреляция Пирсона(по формуле): {pearson_corr:.4f}")
print(f"P-значение: {p_value:.4f}")

# Доверительный интервал через преобразование Фишера
z = 0.5 * np.log((1 + pearson_corr) / (1 - pearson_corr))
se = 1 / np.sqrt(n - 3)
z_lower = z - 1.96 * se
z_upper = z + 1.96 * se

# Обратное преобразование
r_lower = (np.exp(2 * z_lower) - 1) / (np.exp(2 * z_lower) + 1)
r_upper = (np.exp(2 * z_upper) - 1) / (np.exp(2 * z_upper) + 1)

print(f"95% доверительный интервал для корреляции Пирсона: [{r_lower:.3f}, {r_upper:.3f}]")


In [ ]:
from scipy.stats import spearmanr

# Расчет корреляции Спирмена
r_s, p_value_s = spearmanr(df_selected[columns_to_plot[0]], df_selected[columns_to_plot[1]])
print(f"Корреляция Спирмена между {columns_to_plot[0]} и {columns_to_plot[1]}: {r_s:.3f}, p-value: {p_value_s:.3f}")

# Вычисляем ранги
rank_x = x.rank(method='average')
rank_y = y.rank(method='average')

# Разности рангов
d = rank_x - rank_y

# Коэффициент корреляции Спирмена по формуле
spearman_corr = 1 - (6 * np.sum(d**2)) / (n * (n**2 - 1))

# Статистика t
t_stat = spearman_corr * np.sqrt(n - 2) / np.sqrt(1 - spearman_corr**2)

# p-value (двусторонний тест)
p_value = 2 * t.sf(np.abs(t_stat), df=n - 2)

# Вывод
print(f"Корреляция Спирмена (по формуле): {spearman_corr:.4f}")
print(f"P-значение: {p_value:.4f}")


In [ ]:
from scipy.stats import kendalltau

# Расчет корреляции Кенделла
tau, p_value_tau = kendalltau(df_selected[columns_to_plot[0]], df_selected[columns_to_plot[1]])
print(f"Корреляция Кенделла между {columns_to_plot[0]} и {columns_to_plot[1]}: {tau:.3f}, p-value: {p_value_tau:.3f}")

import pandas as pd
import numpy as np
from scipy.stats import norm

# Предполагаем, что df_selected уже создан
x = df_selected['height'].values
y = df_selected['age'].values
n = len(x)

# Считаем количество согласованных и несогласованных пар
C = 0  # согласованные
D = 0  # несогласованные

for i in range(n - 1):
    for j in range(i + 1, n):
        dx = x[i] - x[j]
        dy = y[i] - y[j]
        sign = np.sign(dx * dy)
        if sign > 0:
            C += 1
        elif sign < 0:
            D += 1
        # если sign == 0 → пара не учитывается (связь отсутствует)

# Коэффициент Кенделла
kendall_tau = (C - D) / (0.5 * n * (n - 1))

# Стандартная ошибка и p-value (приближение через нормальное распределение)
se = np.sqrt((2 * (2 * n + 5)) / (9 * n * (n - 1)))
z = kendall_tau / se
p_value = 2 * norm.sf(np.abs(z))

# Вывод
print(f"Корреляция Кенделла (по формуле): {kendall_tau:.4f}")
print(f"P-значение: {p_value:.4f}")



In [ ]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

# Выбираем столбцы
x = df_selected[columns_to_plot[0]].values.reshape(-1, 1)  # независимая переменная
y = df_selected[columns_to_plot[1]].values  # зависимая переменная

# Создаем и обучаем модель
lin_model = LinearRegression()
lin_model.fit(x, y)

# Коэффициенты
w1 = lin_model.coef_[0]   # наклон
w0 = lin_model.intercept_  # смещение
print(f"Уравнение регрессии: y = {w1:.3f}*x + {w0:.3f}")

# Вычисляем коэффициенты линейной регрессии вручную
x_mean = np.mean(x)
y_mean = np.mean(y)

# Наклон (w1)
w1 = np.sum((x - x_mean) * (y - y_mean)) / np.sum((x - x_mean)**2)

# Смещение (w0)
w0 = y_mean - w1 * x_mean

# 2. Уравнение регрессии (вручную)
print(f"Уравнение линейной регрессии (вручную): y = {w1:.4f} * x + {w0:.4f}")

# Построение графика
plt.figure(figsize=(7,5))
plt.scatter(x, y, color='blue', label='Данные')
plt.plot(x, lin_model.predict(x), color='red', label='Линейная регрессия')
plt.xlabel(columns_to_plot[0])
plt.ylabel(columns_to_plot[1])
plt.title(f'Линейная регрессия: {columns_to_plot[1]} ~ {columns_to_plot[0]}')
plt.legend()
plt.show()


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Преобразуем x в квадратичную форму
poly = PolynomialFeatures(degree=2)
x_poly = poly.fit_transform(x)  # создаёт [1, x, x^2]

# Обучаем модель линейной регрессии на квадратичных признаках
quad_model = LinearRegression()
quad_model.fit(x_poly, y)

# Коэффициенты
w0 = quad_model.intercept_
w1, w2 = quad_model.coef_[1], quad_model.coef_[2]
print(f"Квадратичное уравнение регрессии: y = {w2:.3f}*x^2 + {w1:.3f}*x + {w0:.3f}")

# Построение графика
x_range = np.linspace(x.min(), x.max(), 100).reshape(-1, 1)
x_range_poly = poly.transform(x_range)
y_pred = quad_model.predict(x_range_poly)

plt.figure(figsize=(7,5))
plt.scatter(x, y, color='blue', label='Данные')
plt.plot(x_range, y_pred, color='red', label='Квадратичная регрессия')
plt.xlabel(columns_to_plot[0])
plt.ylabel(columns_to_plot[1])
plt.title(f'Квадратичная регрессия: {columns_to_plot[1]} ~ {columns_to_plot[0]}')
plt.legend()
plt.show()


In [ ]:
# Очистим данные от нулей и NaN (x не должен быть 0)
df_clean = df_selected[['height', 'age']].dropna()
df_clean = df_clean[df_clean['height'] != 0]

x = df_clean['height'].values
y = df_clean['age'].values

# Преобразуем x → 1/x
x_trans = (1 / x).reshape(-1, 1)

# Обучаем линейную регрессию на 1/x
hyp_model = LinearRegression()
hyp_model.fit(x_trans, y)

# Получаем параметры
w0 = hyp_model.coef_[0]
w1 = hyp_model.intercept_
print(f"Гиперболическая регрессия: y = {w0:.4f} / x + {w1:.4f}")

# Предсказания
x_range = np.linspace(x.min(), x.max(), 200)
y_pred = w0 / x_range + w1

# Визуализация
plt.figure(figsize=(7, 5))
plt.scatter(x, y, color='blue', label='Данные')
plt.plot(x_range, y_pred, color='red', label='Гиперболическая регрессия')
plt.xlabel('height')
plt.ylabel('age')
plt.title('Гиперболическая регрессия: age ~ 1/height')
plt.legend()
plt.grid(True)
plt.show()




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Удалим строки с нулями и NaN (y должен быть > 0)
df_clean = df_selected[['height', 'age']].dropna()
df_clean = df_clean[df_clean['age'] > 0]

x = df_clean['height'].values.reshape(-1, 1)
y = df_clean['age'].values

# Логарифмируем y
log_y = np.log(y)

# Обучаем линейную модель: ln(y) = a + b * x
deg_model = LinearRegression()
deg_model.fit(x, log_y)

# Получаем параметры
a = deg_model.intercept_
b = deg_model.coef_[0]

# Восстанавливаем w0 и w1
w0 = np.exp(a)
w1 = np.exp(b)

print(f"Показательная регрессия: y = {w1:.4f}^x * {w0:.4f}")

x_range = np.linspace(x.min(), x.max(), 200).reshape(-1, 1)
y_pred = w0 * (w1 ** x_range)

# Визуализация
plt.figure(figsize=(7, 5))
plt.scatter(x, y, color='blue', label='Данные')
plt.plot(x_range, y_pred, color='purple', label='Показательная регрессия')
plt.xlabel('height')
plt.ylabel('age')
plt.title('Показательная регрессия: age ~ w1^x * w0')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
from sklearn.metrics import r2_score

lin_pred = lin_model.predict(x)
quad_pred = quad_model.predict(x_poly)
hyp_pred = hyp_model.predict(x_trans)
deg_pred = deg_model.predict(x) 

models = {
    "Линейная": lin_pred,
    "Квадратичная": quad_pred,
    "Гиперболическая": hyp_pred,
    "Показательная": deg_pred,
}

r2_scores = {name: r2_score(y, pred) for name, pred in models.items()}
best_model = max(r2_scores, key=r2_scores.get)
worst_model = min(r2_scores, key=r2_scores.get)

print(f"Лучшая модель: {best_model}, R² = {r2_scores[best_model]:.4f}")
print(f"Худшая модель: {worst_model}, R² = {r2_scores[worst_model]:.4f}")


In [ ]:
from sklearn.metrics import mean_squared_error
from scipy import stats
import numpy as np

def f_test(y_true, y_pred, num_params):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    df_model = num_params
    df_resid = len(y_true) - num_params - 1
    ms_model = (ss_tot - ss_res) / df_model
    ms_resid = ss_res / df_resid
    F = ms_model / ms_resid
    p_value = 1 - stats.f.cdf(F, df_model, df_resid)
    return F, p_value

# Используем best_model и worst_model из предыдущего шага
F_best, p_best = f_test(y, models[best_model], num_params=2)  # квадратичная: 2 параметра
F_worst, p_worst = f_test(y, models[worst_model], num_params=1)  # показательная: 1 параметр

print(f"Лучшая модель ({best_model}): F = {F_best:.4f}, p-value = {p_best:.4f}")
print(f"Худшая модель ({worst_model}): F = {F_worst:.4f}, p-value = {p_worst:.4f}")
